# Predicting Red Wine Quality, Part 2
Data from http://archive.ics.uci.edu/ml/datasets/Wine+Quality

## Citations
<pre>
Dua, D. and Karra Taniskidou, E. (2017). 
UCI Machine Learning Repository [http://archive.ics.uci.edu/ml/index.php]. 
Irvine, CA: University of California, School of Information and Computer Science.
</pre>

<pre>
P. Cortez, A. Cerdeira, F. Almeida, T. Matos and J. Reis. 
Modeling wine preferences by data mining from physicochemical properties.
In Decision Support Systems, Elsevier, 47(4):547-553. ISSN: 0167-9236.
</pre>

Available at:
- [@Elsevier](http://dx.doi.org/10.1016/j.dss.2009.05.016)
- [Pre-press (pdf)](http://www3.dsi.uminho.pt/pcortez/winequality09.pdf)
- [bib](http://www3.dsi.uminho.pt/pcortez/dss09.bib)

## Setup

In [ ]:
%matplotlib inline

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns

red_wine = pd.read_csv('data/winequality-red.csv')
red_wine['high_quality'] = pd.cut(red_wine.quality, bins=[0, 6, 10], labels=[0, 1])

Since we completed our EDA in the [`red_wine.ipynb`](../lab_09/red_wine.ipynb) notebook for last chapter, we will just look at the first 5 rows to refresh our memory of the data rather than repeating the EDA here.

In [ ]:
red_wine.head()

## Train Test Split
As in chapter 9, we will try to predict which red wines will be high-quality:

In [ ]:
from sklearn.model_selection import train_test_split

red_y = red_wine.pop('high_quality')
red_X = red_wine.drop(columns='quality')

r_X_train, r_X_test, r_y_train, r_y_test = train_test_split(
    red_X, red_y, test_size=0.1, random_state=0, stratify=red_y
)

## Logistic Regression Classification of Red Wine Quality from Chapter 9
This was the result from chapter 9 for reference:

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler

red_quality_lr = Pipeline([
    ('scale', StandardScaler()), 
    ('lr', LogisticRegression(
        class_weight='balanced', random_state=0
    ))
]).fit(r_X_train, r_y_train)

quality_preds = red_quality_lr.predict(r_X_test)

This model needs some tuning:

In [ ]:
from sklearn.metrics import classification_report
print(classification_report(r_y_test, quality_preds))

In [ ]:
from ml_utils.classification import plot_roc

plot_roc(r_y_test, red_quality_lr.predict_proba(r_X_test)[:,1])

In [ ]:
from ml_utils.classification import confusion_matrix_visual

confusion_matrix_visual(r_y_test, quality_preds, ['low', 'high'])

## Searching for the Best Hyperparameters with Plots
We have been working with training and testing sets; however, in order to try out different hyperparameters, we need a third set: the validation set. We will train with the training set as usual. The validation set will be used to test different hyperparameters. Only after we have our model tuned will we test with the testing set. Note that the validation set is not the testing set, nor should they contain the same data. 

One way of making the validation set would be to run `train_test_split()` on the training data:

In [ ]:
from sklearn.model_selection import train_test_split

r_X_train_new, r_X_validate, r_y_train_new, r_y_validate = train_test_split(
    r_X_train, r_y_train, test_size=0.3, random_state=0, stratify=r_y_train
)

`C` is the inverse of the regularization strength. It determines the weight on the penalty term. We will try 10 values from $10^{-1}$ to $10^1$ for `C`. To make this range of numbers, we use `np.logspace()` and provide the exponents (-1 and 1) of the minimum and maximum values in the range. We then get evenly-spaced values in between:

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import f1_score
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import MinMaxScaler

inv_regularization_strengths = np.logspace(-1, 1, num=10)
scores = []

for inv_reg_strength in inv_regularization_strengths:
    pipeline = Pipeline([
        ('scale', MinMaxScaler()),
        ('lr', LogisticRegression(
            class_weight='balanced', random_state=0,
            C=inv_reg_strength
        ))
    ]).fit(r_X_train_new, r_y_train_new)
    scores.append(
        f1_score(pipeline.predict(r_X_validate), r_y_validate)
    )

plt.plot(inv_regularization_strengths, scores, 'o-')
plt.xscale('log')
plt.xlabel('inverse of regularization strength (C)')
plt.ylabel(r'$F_1$ score')
plt.title(r'$F_1$ score vs. Inverse of Regularization Strength')

## Searching for the Best Hyperparameters with `GridSearchCV`
We can specify a search space as a dictionary of parameter names and values to try for each. Note that if we have any preprocessing steps, we must use a pipeline. We can tune hyperparameters in a pipeline if we prefix the hyperparameter name with the step's name followed by `__`. We can specify the metric to use for the tuning as well:

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import MinMaxScaler

pipeline = Pipeline([
    ('scale', MinMaxScaler()),
    ('lr', LogisticRegression(
        class_weight='balanced', random_state=0
    ))
])

search_space = {
    'lr__C': np.logspace(-1, 1, num=10),
    'lr__fit_intercept': [True, False]
}

lr_grid = GridSearchCV(
    pipeline, search_space, scoring='f1_macro', cv=5
).fit(r_X_train, r_y_train)

We can use the `best_params_` attribute to see the best parameters from the grid search:

In [ ]:
lr_grid.best_params_

The `best_score_` shows the score for the specified metric that was achieved on the validation set using the best parameters:

In [ ]:
lr_grid.best_score_

By using the testing set, we can see that our $F_1$ score is now higher than what we got without hyperparameter tuning in chapter 9:

In [ ]:
from sklearn.metrics import classification_report
print(classification_report(r_y_test, lr_grid.predict(r_X_test)))

### `GridSearchCV` with CV Object
We can specify the number of folds to use for cross validation and even change the method of doing so with the `cv` parameter:

In [ ]:
from sklearn.model_selection import RepeatedStratifiedKFold

lr_grid = GridSearchCV(
    pipeline, search_space, scoring='f1_macro',
    cv=RepeatedStratifiedKFold(random_state=0)
).fit(r_X_train, r_y_train)

Note that this is different from before:

In [ ]:
print('Best parameters (CV score=%.2f):\n    %s' % (
    lr_grid.best_score_, lr_grid.best_params_
))

## Polynomial features and interaction terms
We can look at a pairplot to try and find any non-linear relationships between the features in our model:

In [ ]:
sns.pairplot(r_X_train)

If we suspect there is a non-linear relationship between our variables, we can add polynomial features to our model to generalize our linear model. The `PolynomialFeatures` class from scikit-learn will transform our input data into a bias term (1), a term for each of the variables as found in the starting data, a term for each combination of 2 variables multiplied together, and a term for the square of each variable (this can be modified to include higher powers with the `degree` parameter):

In [ ]:
from sklearn.preprocessing import PolynomialFeatures

PolynomialFeatures(degree=2).fit_transform(r_X_train[['citric acid', 'fixed acidity']])

Here is a breakdown of the first row in the previous result to help understand where the numbers came from:

| term | $bias$ | $citric\ acid$ | $fixed\ acidity$ | $citric\ acid^2$ | $citric\ acid \times fixed\ acidity$ | $fixed\ acidity^2$ |
|:---: | :---: | :---: | :---: | :---: | :---: | :---: |
| **value** | 1.000e+00 | 5.500e-01 | 9.900e+00 | 3.025e-01 | 5.445e+00 | 9.801e+01 |

Note we can also specify to not include the bias (`include_bias=False`) and to only give interaction terms (`interaction_only=True`). This leaves us with the value for each of the variables and their interaction term:

In [ ]:
from sklearn.preprocessing import PolynomialFeatures

PolynomialFeatures(
    degree=2, include_bias=False, interaction_only=True
).fit_transform(r_X_train[['citric acid', 'fixed acidity']])

We can put this in a pipeline to build our model with these features:

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import MinMaxScaler, PolynomialFeatures

pipeline = Pipeline([
    ('poly', PolynomialFeatures(degree=2)),
    ('scale', MinMaxScaler()),
    ('lr', LogisticRegression(
        class_weight='balanced', random_state=0
    ))
]).fit(r_X_train, r_y_train)

Notice the performance is slightly better than it was before:

In [ ]:
from sklearn.metrics import classification_report

preds = pipeline.predict(r_X_test)
print(classification_report(r_y_test, preds))

## Feature Unions
We can combine multiple preprocessing transformations on our data with the feature union:

In [ ]:
from sklearn.feature_selection import VarianceThreshold
from sklearn.pipeline import FeatureUnion, Pipeline
from sklearn.preprocessing import MinMaxScaler, PolynomialFeatures
from sklearn.linear_model import LogisticRegression

combined_features = FeatureUnion([
    ('variance', VarianceThreshold(threshold=0.01)),
    ('poly', PolynomialFeatures(degree=2, include_bias=False, interaction_only=True))
])

pipeline = Pipeline([
    ('normalize', MinMaxScaler()),
    ('feature_union', combined_features),
    ('lr', LogisticRegression(
        class_weight='balanced', random_state=0
    ))
]).fit(r_X_train, r_y_train)

See the feature union (first 9 are from `VarianceThreshold`, rest are interaction terms):

In [ ]:
pipeline.named_steps['feature_union'].transform(r_X_train)[0]

This also results in marginal improvements in recall in $F_1$ score:

In [ ]:
from sklearn.metrics import classification_report

preds = pipeline.predict(r_X_test)
print(classification_report(r_y_test, preds))

In [ ]:
from ml_utils.classification import plot_pr_curve
plot_pr_curve(r_y_test, pipeline.predict_proba(r_X_test)[:,1])

In [ ]:
from ml_utils.classification import confusion_matrix_visual
confusion_matrix_visual(r_y_test, preds, ['low', 'high'])

## Ensemble methods
Ensemble methods combine many models (often weak ones) to create another (stronger one) that will either minimize average error between actual and predicted (the bias) or improve how well it generalizes to unseen data (minimize the variance). We have to strike a balance between complex models that may increase variance, as they tend to overfit, with simple models that may have high bias, as they tend to underfit. This is called the bias-variance trade-off, which is illustrated in the following subplots:

In [ ]:
from visual_aids.ml_viz import bias_variance_tradeoff
bias_variance_tradeoff()

### Ensemble Method: Random Forest
A random forest is a bagging technique (bootstrap aggregation), where we build many decision trees that each get a different bootstrapped sample of the data. At the end, this is aggregated by voting for classification and averaging for regression:

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV

rf = RandomForestClassifier(n_estimators=100, random_state=0)

search_space = {
    'max_depth': [4, 8],
    'min_samples_leaf': [4, 6]
}

rf_grid = GridSearchCV(
    rf, search_space, cv=5, scoring='precision'
).fit(r_X_train, r_y_train)

rf_preds = rf_grid.predict(r_X_test)
rf_grid.score(r_X_test, r_y_test)

### Ensemble Method: Gradient Boosted Trees
This is a boosting technique, meaning many weak learners are trained, but each learns from the mistakes of the others:

In [ ]:
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.model_selection import GridSearchCV

gb = GradientBoostingClassifier(n_estimators=100, random_state=0)

search_space = {
    'max_depth': [4, 8],
    'min_samples_leaf': [4, 6],
    'learning_rate': [0.1, 0.5, 1]
}

gb_grid = GridSearchCV(
    gb, search_space, cv=5, scoring='f1_macro'
).fit(r_X_train, r_y_train)

gb_preds = gb_grid.predict(r_X_test)
gb_grid.score(r_X_test, r_y_test)

### Voting
We can combine various models with voting. Often it will be interesting to first check their level of agreement with Cohen's Kappa score. Let's check the agreement between the gradient boosting classifier and the random forest range. This metric has a range of [-1, 1]:

In [ ]:
from sklearn.metrics import cohen_kappa_score
cohen_kappa_score(
    rf_grid.predict(r_X_test), gb_grid.predict(r_X_test)
)

There are two ways to conduct voting:
- majority rules (hard)
- highest probability (soft)

In [ ]:
from sklearn.ensemble import VotingClassifier

majority_rules = VotingClassifier(
    [('lr', lr_grid.best_estimator_), ('rf', rf_grid.best_estimator_), ('gb', gb_grid.best_estimator_)],
    voting='hard'
).fit(r_X_train, r_y_train)

max_probabilities = VotingClassifier(
    [('lr', lr_grid.best_estimator_), ('rf', rf_grid.best_estimator_), ('gb', gb_grid.best_estimator_)],
    voting='soft'
).fit(r_X_train, r_y_train)

Agreement between majority rules and max probabilities:

In [ ]:
cohen_kappa_score(
    majority_rules.predict(r_X_test), max_probabilities.predict(r_X_test)
)

#### Majority Rules Evaluation

This is our best model yet:

In [ ]:
from sklearn.metrics import classification_report
print(classification_report(r_y_test, majority_rules.predict(r_X_test)))

#### Max Probabilities Evaluation

This performs worse than the majority rules voting:

In [ ]:
from sklearn.metrics import classification_report
print(classification_report(r_y_test, max_probabilities.predict(r_X_test)))

## Class Imbalances
k-NN with 5 neighbors for a baseline

In [ ]:
from sklearn.neighbors import KNeighborsClassifier

knn = KNeighborsClassifier(
    n_neighbors=5
).fit(r_X_train, r_y_train)

knn_preds = knn.predict(r_X_test)

k-NN trains fast because it is a **lazy learner** &mdash; calculations are made at classification time. Note times will vary:

In [ ]:
%%timeit
from sklearn.neighbors import KNeighborsClassifier
knn = KNeighborsClassifier(
    n_neighbors=5
).fit(r_X_train, r_y_train)

Compare this to a support vector machine (SVM):

In [ ]:
%%timeit
from sklearn.svm import SVC
knn = SVC(gamma='auto').fit(r_X_train, r_y_train)

Check the performance of the baseline for reference later:

In [ ]:
from sklearn.metrics import classification_report
print(classification_report(r_y_test, knn_preds))

In [ ]:
from ml_utils.classification import plot_pr_curve
plot_pr_curve(r_y_test, knn.predict_proba(r_X_test)[:,1])

In [ ]:
from ml_utils.classification import confusion_matrix_visual
confusion_matrix_visual(r_y_test, knn_preds, ['low', 'high'])

### Random under-sampling
We will under-sample the majority class, which will reduce the amount of training data available:

In [ ]:
from imblearn.under_sampling import RandomUnderSampler

X_train_undersampled, y_train_undersampled = RandomUnderSampler(
    random_state=0
).fit_resample(r_X_train, r_y_train)

Notice how few observations we started with in the minority class:

In [ ]:
r_y_train.value_counts() # before

That is now the number of observations in the majority class. We lost over 50% of the data:

In [ ]:
pd.Series(y_train_undersampled).value_counts().sort_index() # after

Fitting the model is the same as before:

In [ ]:
from sklearn.neighbors import KNeighborsClassifier

knn_undersampled = KNeighborsClassifier(
    n_neighbors=5
).fit(X_train_undersampled, y_train_undersampled)

knn_undersampled_preds = knn_undersampled.predict(r_X_test)

Due to the lack of available data, this model is worse than before:

In [ ]:
from sklearn.metrics import classification_report
print(classification_report(r_y_test, knn_undersampled_preds))

In [ ]:
from ml_utils.classification import plot_pr_curve
plot_pr_curve(r_y_test, knn_undersampled.predict_proba(r_X_test)[:,1])

In [ ]:
from ml_utils.classification import confusion_matrix_visual
confusion_matrix_visual(r_y_test, knn_undersampled_preds, ['low', 'high'])

### Over-sampling with [SMOTE](https://arxiv.org/pdf/1106.1813.pdf)
This technique will make synthetic data, so it's important to if it is a reasonable assumption to make that the data we have is representative of the full spectrum we will see and whether it will change over time:

In [ ]:
from imblearn.over_sampling import SMOTE

X_train_oversampled, y_train_oversampled = SMOTE(
    random_state=0
).fit_resample(r_X_train, r_y_train)

Before, the imbalance could be observed in the training set:

In [ ]:
r_y_train.value_counts() # before

Now, both classes have the same number of observations:

In [ ]:
pd.Series(y_train_oversampled).value_counts().sort_index() # after

Building the model with the oversampled data is the same as before:

In [ ]:
from sklearn.neighbors import KNeighborsClassifier

knn_oversampled = KNeighborsClassifier(
    n_neighbors=5
).fit(X_train_oversampled, y_train_oversampled)

knn_oversampled_preds = knn_oversampled.predict(r_X_test)

This model has higher recall than the class imbalance k-NN model:

In [ ]:
from sklearn.metrics import classification_report
print(classification_report(r_y_test, knn_oversampled_preds))

In [ ]:
from ml_utils.classification import plot_pr_curve
plot_pr_curve(r_y_test, knn_oversampled.predict_proba(r_X_test)[:,1])

In [ ]:
from ml_utils.classification import confusion_matrix_visual
confusion_matrix_visual(r_y_test, knn_oversampled_preds, ['low', 'high'])

<hr>
<div style="overflow: hidden; margin-bottom: 10px;">
    <div style="float: left;">
        <a href="../../lab_09/red_wine.ipynb">
            <button>&#8592; Chapter 9</button>
        </a>
        <a href="./planets_ml.ipynb">
            <button>Planets</button>
        </a>
        <a href="./wine.ipynb">
            <button>Red + White Wine</button>
        </a>
    </div>
    <div style="float: right;">
        <a href="../../solutions/lab_10/exercise_1.ipynb">
            <button>Solutions</button>
        </a>
        <a href="../lab_11/1-EDA_unlabeled_data.ipynb">
            <button>Chapter 11 &#8594;</button>
        </a>
    </div>
</div>
<hr>